# **Hate Speech Detection Using BERT Model**

In [ ]:
import pkg_resources
import subprocess
import pickle
import shutil
import time
import sys
import os

from enum import Enum
from functools import partial

In [ ]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.3 MB/s eta 0:00:00


In [ ]:
!pip install fastapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 12.4 MB/s eta 0:00:00


In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=b328d9b8e51267d5125fe5fc2b9e3af0c6811bdcb3ca4fe97acdf3ad5b6d3b82
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [ ]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 KB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 32.2 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade transformers
!pip install fastapi
!pip install wget
!pip install pytorch-lightning

LIB = {'fastapi','wget','pytorch-lightning',
  'spacy', 'scikit-learn', 'numpy', 'pandas', 'torch', 'transformers'
}

installed = {pkg.key for pkg in pkg_resources.working_set}
missing = LIB - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

from typing import List, Dict, Tuple
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Library Import**

In [ ]:
from dataclasses import dataclass

from sklearn.metrics import accuracy_score

from torch import torch
from torch.utils.data import Dataset
from torch import nn

import numpy as np
import pandas as pd
from pandas import DataFrame

from transformers import PreTrainedTokenizer, PreTrainedModel
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import DistilBertModel, DistilBertPreTrainedModel
from transformers import Trainer, TrainingArguments
from transformers import EvalPrediction

# **Reading  White Supremacy Forum  DataSet**


In [ ]:
# dataset git repo
if not os.path.exists("hate-speech-dataset"):
  !git clone https://github.com/Vicomtech/hate-speech-dataset.git

Cloning into 'hate-speech-dataset'...
remote: Enumerating objects: 10785, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 10785 (delta 6), reused 0 (delta 0), pack-reused 10771
Receiving objects: 100% (10785/10785), 1.16 MiB | 5.42 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [ ]:
def readData(paths: List[str], group: str):
  pairs = []
  for p in paths:

    with open(p) as f:
      file_id = p.split('/')[-1].split('.')[0]
      pairs.append((file_id, f.read(), group))

  return pd.DataFrame(pairs, columns=["file_id", "text", "gSet"])

idLabels = pd.read_csv('hate-speech-dataset/annotations_metadata.csv')[["file_id", "label"]]

trainDf = readData(glob.glob('./hate-speech-dataset/sampled_train/*.txt'), 'train')
testDf = readData(glob.glob('./hate-speech-dataset/sampled_test/*.txt'), 'test')

trainDf = trainDf.join(idLabels.set_index('file_id'), on='file_id')
testDf = testDf.join(idLabels.set_index('file_id'), on='file_id')
# print(trainDf.info(), '\n')
# print(testDf.info())

In [ ]:
trainDf = trainDf.drop('gSet', axis=1)
testDf = testDf.drop('gSet', axis=1)
trainDf = trainDf.drop('file_id', axis=1)
testDf = testDf.drop('file_id', axis=1)

In [ ]:
trainDf

,text,label
0,Great image of the Jew as a poisonous mushroom .,hate
1,"I listen to a lot of metal so , yeah , we call...",noHate
2,You are the loyal punk to dead men that would ...,hate
3,Here is a video of the event that is worth wat...,hate
4,"If they do that , they can do what they want i...",noHate
...,...,...
1909,All the jews I have ever known will do anythin...,hate
1910,I'm insulted by how stupid the Jews think we a...,hate
1911,I'm sad to find this out right now but I 'm al...,noHate
1912,South Africa has returned to the Stone Age in ...,hate


# **Reading  Ethos-Hate-Speech-Dataset  DataSet**


In [ ]:
!git clone https://github.com/intelligence-csd-auth-gr/Ethos-Hate-Speech-Dataset.git



Cloning into 'Ethos-Hate-Speech-Dataset'...
remote: Enumerating objects: 303, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 303 (delta 26), reused 1 (delta 1), pack-reused 261
Receiving objects: 100% (303/303), 12.68 MiB | 17.89 MiB/s, done.
Resolving deltas: 100% (176/176), done.


In [ ]:
import os
import pandas as pd

dataset_path = '/content/Ethos-Hate-Speech-Dataset/ethos/ethos_data/Ethos_Dataset_Binary.csv'

# Example usage: load the train set
train_data_path = os.path.join(dataset_path)
df = pd.read_csv(train_data_path,delimiter=';')

In [ ]:
df["isHate"] = df["isHate"].apply(lambda x: "noHate" if x < .5 else "hate")
df['label'] = df['isHate'].replace({0: 'hate', 1: 'noHate'})
df = df.rename(columns={'comment': 'text'})
df = df.drop('isHate', axis =1)

In [ ]:
from sklearn.model_selection import train_test_split

# train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
train_df, test_df = train_test_split(df, test_size=0.3, stratify=df['label'])

In [ ]:
################Code to run just this data set#####################################
trainDf = train_df
testDf = test_df


In [ ]:
# trainDf = pd.concat([trainDf, train_df])
# testDf = pd.concat([testDf, test_df])
# trainDf = trainDf.reset_index(drop=True)
# testDf = testDf.reset_index(drop=True)

NameError: ignored

In [ ]:
trainDf = trainDf.reset_index(drop=True)
testDf = testDf.reset_index(drop=True)
trainDf

,text,label
0,"seriously, any arab or turks i've ever met had...",hate
1,why am i watching this,noHate
2,"Ugh, my heart hurt for her when she was talkin...",noHate
3,Very sad for those people. i was a young boy i...,noHate
4,Why aren't Europe evicting those bastards ru...,hate
...,...,...
693,Women are equally as strong as men. They sh...,noHate
694,The thing is women are not equal to us men and...,hate
695,"The male gene is stronger, as it should be of ...",hate
696,We'll walk right up to the sun hand in hand..,noHate


# **Reading AAAI Conference on Web and Social Media Thomas Davidson DataSet**


In [ ]:
import pandas as pd
import urllib.request

url = 'https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv'
file_name = 'labeled_data.csv'


In [ ]:
# download file
urllib.request.urlretrieve(url, file_name)

# load into dataframe
df = pd.read_csv(file_name)

# print first 5 rows
print(df.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
import re
import string
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# load data into a dataframe
df = pd.read_csv('labeled_data.csv')

# define preprocessing function
def preprocess_text(text):
    # remove usernames
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)

    # remove URLs
    text = re.sub(r'http\S+', '', text)

    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # convert to lowercase
    text = text.lower()

    # tokenize text into words
    words = word_tokenize(text)

    # remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]

    # lemmatize words
    lem = WordNetLemmatizer()
    words = [lem.lemmatize(word) for word in words]

    # join words back into a string
    text = ' '.join(words)

    return text

# apply preprocessing to 'tweet' column
df['tweet'] = df['tweet'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
df1=df
df1['class'] = df['class'].replace({0: 'hate',1: 'offensive', 2: 'noHate'})

In [ ]:
new_df = df[['tweet', 'class']].rename(columns={'tweet': 'text', 'class': 'label'}).copy()

In [ ]:
new_df = new_df[new_df['label'] != 'offensive']
# new_df = new_df.reset_index(drop=True)

In [ ]:
from sklearn.model_selection import train_test_split

train_n, test_n = train_test_split(new_df, test_size=0.3, stratify=new_df['label'])

In [ ]:
############################# Just for this data set##################################
# trainDf = train_n
# testDf = test_n

In [ ]:
trainDf = pd.concat([trainDf, train_n])
testDf = pd.concat([testDf, test_n])

In [ ]:
trainDf = trainDf.reset_index(drop=True)
testDf = testDf.reset_index(drop=True)

# **Bert Model For Distilbert**

In [ ]:


tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
# save original model set of weights. They we'll be used later over multiple iterations
torch.save(model.state_dict(), "./tempBertState.pt")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

# **Bert Model For bert-base-uncased**

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
# model.to(device)

from transformers import BertTokenizer, BertForSequenceClassification, AdamW


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# save original model set of weights. They we'll be used later over multiple iterations
torch.save(model.state_dict(), "./tempBertState.pt")



KeyboardInterrupt: ignored

In [ ]:
@dataclass
class Triple:
  """
  Class for keeping track of single tokenized inputs
  (for Pytorch Dataset __getitem__).
  """
  inputs: List[int]
  masks: List[int]
  label: int

  def triple(self):
    return (self.inputs, self.masks, self.label)


class HateDataset(Dataset):
  """PyTorch Dataset class. Required as Trainer input."""
  def __init__(self,
    corpus: List[str], labels: List[str],
    tokenizer: PreTrainedTokenizer=tokenizer,
    truncate=True,
    padding=True
  ):
    self.tokenizer = tokenizer
    self.trunc = truncate
    # corpus is entirely tokenized during instantiation
    self.corpusTokenized = tokenizer(corpus, padding=padding)
    self.labels = labels


  def __getitem__(self, i):
    inputs = self.corpusTokenized['input_ids'][i]
    masks = self.corpusTokenized['attention_mask'][i]

    return Triple(
      self._truncate(inputs) if self.trunc else inputs,
      self._truncate(masks, is_mask=True) if self.trunc else masks,
      self.labels[i]
    )

  def __len__(self):
    return len(self.labels)

  def _truncate(self, vector: List[int], is_mask=False, sep_token_num=102):
    """
    Truncate vector length to model max input size (the default transformers
    library implementation does not seem to work with batches!).
    """
    limit = self.tokenizer.model_max_length

    if len(vector) <= limit:
      return vector
    elif is_mask:
      return vector[:limit]
    else:
      last = sep_token_num if vector[-1] not in (0, sep_token_num) else 0
      return vector[:limit - 1] + [last]


def collator(items: List[Triple]) -> Dict:
  """
  Collects list of items into a single dictionary; this dictionary's keys/values match
  the parameter names and format of the model selected. This function is used by the
  Pytorch Dataloader inside the Trainer class.
  """
  # keys for DistilBertForSequenceClassification
  input_k = 'input_ids'
  mask_k = 'attention_mask'
  label_k = 'labels'

  # zipps all input_ids, attention_mask and labels together
  zipped =  list(map(list, zip(*[t.triple() for t in items])))
  # convert lists to pytorch tensors
  inputs, masks, labels = list(map(lambda ls: torch.tensor(ls), zipped))

  return {input_k: inputs, mask_k: masks, label_k: labels}


def prepareDf(df: DataFrame, textCol='text', labelsCol='label'):
  """Extract relevant columns to be fed to a Dataset object."""
  return {
    "corpus": df[textCol].tolist(),
    "labels": df[labelsCol].map({'hate': 1, 'noHate': 0})
  }


def hateMetrics(pred: EvalPrediction):
  """
  Compute 3 accuracies: all labels, hate, noHate. These are the baseline metrics
  used in the paper to compare models (https://arxiv.org/pdf/1809.04444.pdf, page 7).
  Function assumes 'hate' group uses label 1, 'noHate' uses label 0.

  For reference, results from paper:

      Accu | hate | noHate | all
      ---------------------------
      SVM  | 0.69 | 0.73   | 0.71
      CNN  | 0.55 | 0.79   | 0.66
      LSTM | 0.71 | 0.75   | 0.73
  """
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)

  mapper = lambda n: True if n == 1 else False

  hatePreds = preds[[mapper(l) for l in labels]]
  noHatePreds = preds[[mapper(l) for l in 1 - labels]]

  return {
      'accHate': accuracy_score(hatePreds, np.ones_like(hatePreds)),
      'accNoHate': accuracy_score(noHatePreds, np.zeros_like(noHatePreds)),
      'accAll': accuracy_score(labels, preds),
  }

In [ ]:
train_dataset = HateDataset(**prepareDf(trainDf))
test_dataset = HateDataset(**prepareDf(testDf))

train_ds_mock = HateDataset(**prepareDf(trainDf[:20]))
test_ds_mock = HateDataset(**prepareDf(testDf[:20]))

Token indices sequence length is longer than the specified maximum sequence length for this model (889 > 512). Running this sequence through the model will result in indexing errors


# **Setting Training Argument**

For embArgs= trainingArgs(2, trainDir)

In [ ]:
def trainingArgs(
  epochs: int,
  trainDir: str,
  batchSizeTrain=16,
  batchSizeEval=32,
  training_set_len=len(train_dataset)
):
  totalSteps = int((training_set_len / batchSizeTrain) * epochs)
  warmup = int(totalSteps * 0.05)

  return TrainingArguments(
    output_dir=f"./{trainDir}/results",
    logging_dir=f"./{trainDir}/logs",

    overwrite_output_dir=True,

    per_device_train_batch_size=batchSizeTrain,
    per_device_eval_batch_size=batchSizeEval,

    num_train_epochs=epochs,
    warmup_steps=warmup,

    logging_steps=9999,
    save_steps=9999,
    save_total_limit=1,

    learning_rate=5e-5,
    weight_decay=1e-2,
  )

In [ ]:
# training arguments
trainDir = "training"
# embArgs = trainingArgs(2, trainDir, 2, 4, len(train_ds_mock))
saveModelDir = "tuned-bert"
# epochsList = [2]
epochsList = [2, 3, 4]
# epochsList = [2]

embArgs= trainingArgs(2, trainDir)
trainDs = train_dataset
testDs = test_dataset

# uncomment these to pass arguments for testing purposes


# trainDs = train_ds_mock
# testDs = test_ds_mock

In [ ]:


from transformers import Trainer, TrainingArguments
from pytorch_lightning.loggers import TensorBoardLogger
import shutil

finalMetrics = list()

for epoch in epochsList:
  model.load_state_dict(torch.load("./tempBertState.pt"))

  embArgs.num_train_epochs = epoch

  trainDir = "train-" + str(epoch)

  logger = TensorBoardLogger("tb_logs", name=trainDir)
  trainer = Trainer(
    model=model,
    args=embArgs, #
    data_collator=collator,
    train_dataset=trainDs,
    eval_dataset=testDs,
    compute_metrics=hateMetrics
  )

  trainer.train()
  evaMetrics: Dict = trainer.evaluate()
  trainLoss: float = trainer.evaluate(trainDs)['eval_loss']

  finalMetrics.append(
    {"no_epoch": epoch, "eval_train_loss_val": trainLoss, **evaMetrics}
  )

  # clean logs
  # shutil.rmtree(f"./{trainDir}")


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


# **Output Evauation Matrix**

In [ ]:
metricsTunedDf = pd.DataFrame(finalMetrics)
metricsTunedDf

,no_epoch,eval_train_loss_val,eval_loss,eval_accHate,eval_accNoHate,eval_accAll,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,2,0.240291,0.485266,0.692308,0.829412,0.770000,5.4984,54.561,1.819,2.0
1,3,0.083174,0.521428,0.684615,0.894118,0.803333,5.1639,58.096,1.937,3.0
2,4,0.020897,0.709491,0.700000,0.894118,0.810000,5.1669,58.062,1.935,4.0


In [ ]:
metricsTunedDf.to_excel('output_for_T_devidson_bert2.xlsx', index=False)

New OutPut

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import PreTrainedModel
from torch.nn import Softmax
import torch

from fastapi import HTTPException

from typing import List, Union
import zipfile
import glob
import wget
import os

PRE_TRAINED_MODEL = "distilbert-base-uncased"
MODEL_DIR = f"{os.getcwd()}/bertEp2"
MODEL_S3_URL = "https://" \
               "cc6e750869d1bf4c575d93c62ceaffbd880f62fdc70d92005eedad24f5865" \
               ".s3.amazonaws.com/bertEp2.zip"

ResponsePair = (List[List[float]], List[List[float]])


class HateClassifier:
    def __init__(self, model_dir=MODEL_DIR, pre_trained=PRE_TRAINED_MODEL):
        self.model: Union[Exception, PreTrainedModel] = self._init_model(model_dir)
        self.tokenizer = AutoTokenizer.from_pretrained(pre_trained)
        self.probaLayer = Softmax(-1)

    def process(self, text: List[str]) -> ResponsePair:
        """Return binary predictions and embeddings from CLS token"""
        tokenized = self.tokenizer(text, padding=True, return_tensors='pt', truncation=True)
        out = self.model(**tokenized, output_hidden_states=True)

        probas, embeds = Softmax(-1)(out[0]), out[1][-1][:, 0]

        return probas.tolist(), embeds.tolist()

    def process_f(self, text: List[str], emb_size=768) -> ResponsePair:
        """Generate random probabilities and embeddings. Used for testing."""
        s = len(text)
        probas = torch.rand([s, 2]).tolist()
        embeddings = torch.rand([s, emb_size]).tolist()

        return probas, embeddings

    def _init_model(self, model_dir: str):
        self._download_model()

        try:
            return AutoModelForSequenceClassification.from_pretrained(model_dir)
        except:
            return HTTPException(status_code=501, detail="Unable to load model.")

    def _download_model(self, model_dir=MODEL_DIR, url=MODEL_S3_URL):
        if any(glob.glob(model_dir + "/*.bin")):
            return

        not os.path.isdir(model_dir) and os.mkdir(model_dir)
        model_path = wget.download(url=url, out=model_dir)

        with zipfile.ZipFile(model_path, 'r') as zip_ref:
            zip_ref.extractall(model_dir)

        os.remove(model_path)

In [ ]:
cls_model = HateClassifier()

In [ ]:
text = "the school board is completely blinded to the events that go on with their schools ... i contacted them about the boy with the knife n they were never even informed of the situation nor were the authorities called , eventhough that is what the school policy requires in such situations ...	"
probas, embeds = cls_model.process(text)


In [ ]:
probas

[[0.4553206264972687, 0.5446794629096985]]

In [ ]:
text = "Japanese culture is ancient and unique	"
# text = "the school board is completely blinded to the events that go on with their schools ... i contacted them about the boy with the knife n they were never even informed of the situation nor were the authorities called , eventhough that is what the school policy requires in such situations ...	"
	# If lies were money he would be the richest negro in the world	hate
# Or are the CIA and FSB on the same team ?
probas1, embeds = cls_model.process(text)

In [ ]:
probas1

[[0.45040959119796753, 0.5495904088020325]]

# **Output Prediction**

In [ ]:
import torch

# Load the trained model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.load_state_dict(torch.load("./tempBertState.pt"))

# Tokenize the input text
text = "Aren’t you ashamed about your body colour?	"
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
inputs = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors="pt")

# Make a prediction
outputs = model(inputs["input_ids"], inputs["attention_mask"])
predicted_label = torch.argmax(outputs[0])
print(predicted_label.argmax().item())

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

0


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load the trained model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.load_state_dict(torch.load("./tempBertState.pt"))

# Tokenize the input text
text = "Aren’t you ashamed about your black body colour?"
# text = "I love you"
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
inputs = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors="pt")

# Make a prediction
outputs = model(inputs["input_ids"], inputs["attention_mask"])
predicted_probs = torch.nn.functional.softmax(outputs[0], dim=1)

# Print the predicted probabilities for both classes
print(predicted_probs)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

tensor([[0.5476, 0.4524]], grad_fn=<SoftmaxBackward0>)


# **Demo Trial Section**

In [ ]:
# import pandas as pd
# import torch
# import os
# from sklearn.model_selection import train_test_split
# from transformers import BertTokenizer, BertForSequenceClassification, AdamW
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

# # dataset git repo
# if not os.path.exists("hate-speech-and-offensive-language"):
#   !git clone https://github.com/t-davidson/hate-speech-and-offensive-language.git

Cloning into 'hate-speech-and-offensive-language'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 41 (delta 4), reused 0 (delta 0), pack-reused 32
Unpacking objects: 100% (41/41), 3.56 MiB | 6.07 MiB/s, done.


In [ ]:

# # Check if GPU is available and select device accordingly
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# # Load dataset
# df = pd.read_csv('/content/hate-speech-and-offensive-language/data/labeled_data.csv', index_col=0)
# df['label'] = df['class'].map({1: 1, 0: 1, 2: 0})  # Convert labels to binary
# # Replace missing or null values with a default value
# # df.fillna(value='unknown', inplace=True)
# # Remove instances with missing or null values
# df.dropna(inplace=True)


# # Split into train and test sets
# train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# # Initialize BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
# model.to(device)



# # Tokenize inputs
# train_encodings = tokenizer(train_df['tweet'].tolist(), truncation=True, padding=True)
# test_encodings = tokenizer(test_df['tweet'].tolist(), truncation=True, padding=True)

# # Convert inputs and labels to PyTorch tensors
# train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']).to(device),
#                               torch.tensor(train_encodings['attention_mask']).to(device),
#                               torch.tensor(train_df['label'].tolist()).to(device))

# test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']).to(device),
#                              torch.tensor(test_encodings['attention_mask']).to(device),
#                              torch.tensor(test_df['label'].tolist()).to(device))

# # # Convert inputs and labels to PyTorch tensors
# # train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
# #                               torch.tensor(train_encodings['attention_mask']),
# #                               torch.tensor(train_df['label'].tolist()))

# # test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
# #                              torch.tensor(test_encodings['attention_mask']),
# #                              torch.tensor(test_df['label'].tolist()))

# # Initialize data loaders
# train_loader = DataLoader(train_dataset, batch_size=16, sampler=RandomSampler(train_dataset))
# test_loader = DataLoader(test_dataset, batch_size=16, sampler=SequentialSampler(test_dataset))

# # Set up optimizer and learning rate scheduler
# optimizer = AdamW(model.parameters(), lr=5e-5)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)



# # Train model
# epochs = 3
# for epoch in [2, 3]:
#     model.train()
#     for batch in train_loader:
#         inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
#         optimizer.zero_grad()
#         outputs = model(**inputs)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         scheduler.step()

#     # Evaluate model on test set
#     model.eval()
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for batch in test_loader:
#             inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
#             outputs = model(**inputs)
#             predictions = torch.argmax(outputs.logits, dim=1)
#             correct += (predictions == batch[2]).sum().item()
#             total += len(batch[2])
#     accuracy = correct / total
#     print(f"Epoch {epoch+1}: Test Accuracy = {accuracy:.3f}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

KeyboardInterrupt: ignored